# Implementación de un modelo de clasificación KNN para predecir la calidad de vinos según sus caracteristicas quimicas. El objetivo es clasificar los vinos en 2 categorias:

- Calidad baja (0): Vinos con puntuación menor a 7
- Calidad alta (1): Vinos con puntuación mayor a 7

El dataset presenta un balance donde los vinos con buena calidad son muy pocos:

0 :   0.86429

1 :   0.13571

In [9]:
import pandas as pd
import numpy as np
# from scikitlearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
df= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/refs/heads/main/winequality-red.csv',delimiter=";")

In [11]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
# Ver info general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [13]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


## KNN para clasificación

In [14]:
# Convertir calidad en variable binaria
df['quality_binary'] = df['quality'].apply(lambda x: 1 if x >= 7 else 0)
# Separar variables predictoras y target
X = df.drop(['quality', 'quality_binary'], axis=1)
y = df['quality_binary']


In [15]:
print(y.value_counts(normalize=True))

quality_binary
0    0.86429
1    0.13571
Name: proportion, dtype: float64


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [13]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[268   9]
 [ 26  17]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       277
           1       0.65      0.40      0.49        43

    accuracy                           0.89       320
   macro avg       0.78      0.68      0.72       320
weighted avg       0.88      0.89      0.88       320



## Optimización de hiperparametro con gridsearch

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': range(1, 31)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train_scaled, y_train)

print("Mejor k:", grid.best_params_)
print("Mejor score:", grid.best_score_)

Mejor k: {'n_neighbors': 1}
Mejor score: 0.5408835139913306


In [20]:
print("Mejor k:", grid.best_params_)
print("Mejor F1-score en validación cruzada:", grid.best_score_)

Mejor k: {'n_neighbors': 1}
Mejor F1-score en validación cruzada: 0.5408835139913306


In [23]:
from sklearn.metrics import classification_report, confusion_matrix
best_knn = grid.best_estimator_
y_pred_best = best_knn.predict(X_test_scaled)
print(classification_report(y_test, y_pred_best))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       277
           1       0.67      0.65      0.66        43

    accuracy                           0.91       320
   macro avg       0.81      0.80      0.80       320
weighted avg       0.91      0.91      0.91       320



In [24]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_best)
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[263  14]
 [ 15  28]]


En la matriz de confusión solo se presentan 14 falsos positivos de un total de 277 casos. Es un modelo confiable para identificar vinos de calidad baja (95% de exactitud) por lo que si se requiere identificar con mayor exactitud vinos de alta calidad, se tienen que implementar técnicas más avanzadas. De un total de 43 vinos de alta calidad, solo detecta 28 de forma correcta. 